# Instructions for setting up JupyterLab to execute C# code

1. **Install .NET SDK:**
   Ensure you have the .NET SDK installed on your machine. You can download it from the [.NET website](https://dotnet.microsoft.com/download).

2. **Install .NET Interactive:**
   Install the .NET Interactive tool using the following command:
   ```bash
   dotnet tool install --global Microsoft.dotnet-interactive
   ```

3. **Install JupyterLab:**
   If you haven't installed JupyterLab yet, use the following command:
   ```bash
   pip install jupyterlab
   ```

4. **Install .NET Kernel for Jupyter:**
   With the .NET Interactive tool installed, set up the .NET kernel by running:
   ```bash
   dotnet interactive jupyter install
   ```

5. **Launch JupyterLab:**
   Start JupyterLab with the following command:
   ```bash
   jupyter lab
   ```

6. **Create a New C# Notebook:**
   In the JupyterLab interface, you should see an option to create a new notebook using the C# (.NET Interactive) kernel.

These steps will allow you to write and execute C# code within JupyterLab, benefiting from its modern features and interface.

Is there anything specific you would like to explore or need further assistance with?

C# Playwright API Reference:  
https://playwright.dev/dotnet/docs/api/class-playwright

In [1]:
// brew install --cask powershell //to install powershell if necessary

The below script needs to be able to find the current output cell; this is an easy method to get it.

In [2]:
// equivalent to `dotnet add package Microsoft.Playwright` if you are developing a .NET project outside of Jupyter Notebooks
// #r "nuget: Microsoft.Playwright"
#r "nuget: Microsoft.Playwright, 1.45.1"
//necessary for including the playwright like so: using Microsoft.Playwright;

Installed Packages Microsoft.Playwright, 1.45.1

If you were not using Jupyter notebooks you would do:

`dotnet add package Microsoft.Playwright`

In command line you need to execute:

    npm install -g playwright
    playwright install

second command installs the browsers which is what you really need

In [3]:
using Microsoft.Playwright;

//necessary for using the assertions like so: Expect(page).NotNull();
using static Microsoft.Playwright.Assertions; // Import static members

using System.Linq;
using System.Text.RegularExpressions;
using System.IO;

In [4]:
void assert(bool condition, string message = "Assertion failed") {
  // Because this does not work in System.Diagnostics.Debug.Assert(15 == 15);
  if (!condition)
  {
    throw new Exception(message);
  }
}

In [5]:
//Hello world
/*
using System.Threading;
var playwright = await Playwright.CreateAsync();
var browser = await playwright.Chromium.LaunchAsync(new BrowserTypeLaunchOptions { Headless = false });
var page = await browser.NewPageAsync();
await page.GotoAsync("https://example.com");
assert(await page.TitleAsync() == "Example Domain");
Thread.Sleep(2000);
await browser.CloseAsync();
//*/

In [6]:
// it seems that in C# there is no Sync version of the Playwright API and it is in general discouraged

# Playwright list
- Browser > Context > Page
- Visit url
- find by text
- find by css
- find by xpath
- find multiple elements and count them
- find inside an element
- visit parent
- visit ancestor
- Reload page
- get text
- clicking
- typing
- mouse hover
- right clicking with context menu
- double clicking
- drag n drop
- pressing tab key
- pressing enter key
- submitting a form
- selecting native dropdown
- getting attribute
- setting attribute
- dismiss alert
- confirm alert press ok
- confirm alert press cancel
- insert for prompt alert
- visit an iframe
- visit a nested iframe
- open new tab
- visit new tab
- open new window
- visit new window  
- upload a file
- download a file
- set browser geolocation
- Go back/forward on pages
- Cookies

============

- return to main frame <--- Selenium only
- return to main window <--- Selenium only
- visit original tab <-- Selenium only

# Configuring Viewport Size and Global Timeout

In [7]:
var playwright = await Playwright.CreateAsync();
var browser = await playwright.Chromium.LaunchAsync(new BrowserTypeLaunchOptions { Headless = false });

//better exploit context than launching a new page directly
// var page = await browser.NewPageAsync();

var context = await browser.NewContextAsync(new BrowserNewContextOptions { 
  ViewportSize = new ViewportSize { Width = 1920, Height = 1080 } 
});

context.SetDefaultTimeout(5000);

var page = await context.NewPageAsync();

// Console.WriteLine(await page.TitleAsync());

In [8]:
// await browser.CloseAsync();

# I visit a url

In [9]:
await page.GotoAsync("https://qaplayground.dev/apps/dynamic-table/");

# I find elements by text and get their text
Also compare innerText with text_content

In [10]:
var loc = page.GetByText("superhero", new() { Exact = false });
var inner_text = await loc.InnerTextAsync(); //how it is rendered on the website
var txt = await loc.TextContentAsync(); //how it is rendered on the website
assert(txt != inner_text);
var column_name = await page.GetByText("superhero", new() { Exact = false }).InnerTextAsync();
assert(column_name == "SUPERHERO");

# I find elements by CSS selector

In [11]:
var top_title = page.Locator("a[aria-label='full courses']");
await Expect(top_title).ToHaveTextAsync("Apps");

# I find elements by XPath selector

In [12]:
var top_right_title = page.Locator("//*[contains(text(), 'View Test Suite')]");
await Expect(top_right_title).ToBeVisibleAsync();

# I find multiple elements and count them

In [13]:
var elems = page.Locator("tr > td");
assert(await elems.CountAsync() == 24);
await Expect(elems).ToHaveCountAsync(24);

# I find an element inside an element

In [14]:
var first_row = page.Locator("thead > tr:nth-of-type(1)");
assert(await first_row.CountAsync() == 1);
var second_header = first_row.Locator("th:nth-of-type(2)");
await Expect(second_header).ToContainTextAsync("Status");

# I get parent of element

In [15]:
// var superhero = page.GetByText("Spider-Man", new PageGetByTextOptions { Exact = true });
var superhero = page.GetByText("Spider-Man", new() { Exact = true }); //here type is inferred
assert(await superhero.IsVisibleAsync());
var parent = superhero.Locator(".."); //using xpath
var email = parent.GetByText("@avengers.com");
await Expect(email).ToHaveTextAsync("spider-man@avengers.com");

# I get ancestor of element

In [16]:
var spiderman = page.GetByText("Spider-Man", new() {Exact=true});
var spiderman_container = spiderman.Locator("//ancestor::tr");
var spiderman_realname_col = spiderman_container.Locator("td:nth-of-type(3)");
assert(await spiderman_realname_col.CountAsync() == 1);
assert((await spiderman_realname_col.TextContentAsync()).Contains("Peter Parker"));

# I reload the page

In [17]:
await page.ReloadAsync();

# I can click and I can type

In [18]:
await page.GotoAsync("https://qaplayground.dev/apps/verify-account/");

In [19]:
await Expect(page.Locator("#title")).ToHaveTextAsync("Verify Your Account");

In [20]:
var nums = Enumerable.Range(1, 6).Select(num => page.Locator($".code-container input:nth-of-type({num})")).ToList();

In [21]:
assert(nums.Count == 6);

In [22]:
var code_regex = new Regex( string.Join('-', Enumerable.Repeat(@"\d", 6)) );

In [23]:
assert( code_regex.Match(" The confirmation code is 9-9-9-9-9-9 ").Value == "9-9-9-9-9-9" );

In [24]:
var phrase = await page.Locator(".info").InnerTextAsync();

In [25]:
var code_nums = code_regex.Match(phrase).Value.Split('-');
code_nums

[ 9, 9, 9, 9, 9, 9 ]

In [26]:
assert(code_nums.Length == 6);

In [27]:
Enumerable.Range(0, 6).ToList()

[ 0, 1, 2, 3, 4, 5 ]

In [28]:
foreach(var ii in Enumerable.Range(0, 6)) {
  // Console.WriteLine(ii);
  await nums[ii].ClickAsync();
  // await nums[ii].FillAsync(code_nums[ii]); //this did not work to trigger the input event
  await nums[ii].PressSequentiallyAsync(code_nums[ii]);
  await Expect(nums[ii]).ToHaveValueAsync(code_nums[ii]);
  await page.WaitForTimeoutAsync(100); //msec //this seems important here
}

In [29]:
await Expect(page.Locator(".info.success")).ToHaveTextAsync("Success");

# I can execute a real hover with the mouse

In [30]:
await page.GotoAsync("https://qaplayground.dev/apps/mouse-hover/");

In [31]:
await Expect(page.Locator(".poster")).ToBeVisibleAsync();

In [32]:
var target_elem = ".title-container";
var target_css_property = "opacity";

In [33]:
await Expect(page.Locator(target_elem)).ToHaveCSSAsync("opacity", "0");

In [34]:
await page.Locator(".poster").HoverAsync();

In [35]:
await Expect(page.Locator(target_elem)).ToHaveCSSAsync(target_css_property, "1");

# I can right click and interact with a context menu

In [36]:
await page.GotoAsync("https://qaplayground.dev/apps/context-menu/");

In [37]:
assert(!await page.Locator("i.uil-trash-alt").IsVisibleAsync());

In [38]:
await page.Locator("#msg").ClickAsync(new() { Button = MouseButton.Right });

In [39]:
assert(await page.Locator("i.uil-trash-alt").IsVisibleAsync());

# I can double click and the action to be recorded as so

In [40]:
await page.GotoAsync("https://play1.automationcamp.ir/mouse_events.html");

In [41]:
assert(!await page.Locator("#click_type").IsVisibleAsync());

In [42]:
await page.Locator("#click_area").ClickAsync(new() { ClickCount = 2 });

In [43]:
assert(await page.Locator("#click_type").IsVisibleAsync());

In [44]:
assert(await page.Locator("#click_type").TextContentAsync() == "Double-Click");

# I drag n drop an element on another element

In [45]:
var drag_elem = page.Locator("#drag_source");
var drop_elem = page.Locator("#drop_target");

In [46]:
await Expect(drop_elem).ToBeVisibleAsync();

In [47]:
await Expect(drop_elem).ToHaveTextAsync("Target");

In [48]:
await Expect(drag_elem).ToHaveTextAsync("Drop me on to the green box");

In [49]:
await page.DragAndDropAsync("#drag_source", "#drop_target");

In [50]:
await Expect(drop_elem).ToHaveTextAsync("Drop is successful!");

# I use tab key and enter key to fill and submit a form

In [51]:
await page.GotoAsync("https://play1.automationcamp.ir/forms.html");

In [52]:
await Expect(page.Locator("#validationCustom04")).Not.ToBeFocusedAsync();

In [53]:
await page.FocusAsync("#validationCustom03");

In [54]:
await page.Locator("#validationCustom03").FillAsync("beautiful city");

In [55]:
await page.PressAsync("#validationCustom03", "Tab");

In [56]:
await Expect(page.Locator("#validationCustom04")).ToBeFocusedAsync();

In [57]:
await page.Locator("#validationCustom04").FillAsync("MaState");

In [58]:
await Expect(page.Locator("#validationCustom05")).Not.ToBeFocusedAsync();

In [59]:
await page.PressAsync("#validationCustom04", "Tab");

In [60]:
await Expect(page.Locator("#validationCustom05")).ToBeFocusedAsync();

In [61]:
var loc = page.Locator("#invalid_terms");
await Expect(loc).ToContainTextAsync("You must agree before submitting");

In [62]:
await Expect(loc).Not.ToBeVisibleAsync();

In [63]:
await page.Locator("#validationCustom05").FillAsync("53479");

In [64]:
await page.PressAsync("#validationCustom05", "Enter");

In [65]:
await Expect(loc).ToBeVisibleAsync();

# I submit a form using native browser submit function

In [66]:
var form_loc = "form.needs-validation";

In [67]:
await page.Locator($"{form_loc} .form-check-input").CheckAsync();

In [68]:
await Expect(page.Locator("#validationCustom03")).ToHaveValueAsync("beautiful city");

In [69]:
await page.EvaluateAsync($"document.querySelector(\"{form_loc}\").submit()");


warning CS1701: Assuming assembly reference 'System.Text.Json, Version=6.0.0.0, Culture=neutral, PublicKeyToken=cc7b13ffcd2ddd51' used by 'Microsoft.Playwright' matches identity 'System.Text.Json, Version=8.0.0.0, Culture=neutral, PublicKeyToken=cc7b13ffcd2ddd51' of 'System.Text.Json', you may need to supply runtime policy



In [70]:
await Expect(page.Locator("#validationCustom03")).ToHaveValueAsync("");

# I select a native dropdown from a form

In [71]:
var loc = page.Locator("#select_tool");

In [72]:
var assertion_loc = page.Locator("#select_tool_validate");

In [73]:
assert(await loc.IsVisibleAsync());

In [74]:
assert(await assertion_loc.IsHiddenAsync());

In [75]:
await Expect(assertion_loc).ToHaveTextAsync("");

In [76]:
await loc.SelectOptionAsync("cyp");

In [77]:
assert(await assertion_loc.IsVisibleAsync());

In [78]:
await Expect(assertion_loc).ToContainTextAsync("cyp");

# I get and I set attributes from an html element

In [79]:
var css_sel = "#check_java";
var loc = page.Locator(css_sel);

In [80]:
await Expect(loc).ToHaveAttributeAsync("disabled", "");

In [81]:
await page.EvaluateAsync($"document.querySelector('{css_sel}').removeAttribute('disabled')");


warning CS1701: Assuming assembly reference 'System.Text.Json, Version=6.0.0.0, Culture=neutral, PublicKeyToken=cc7b13ffcd2ddd51' used by 'Microsoft.Playwright' matches identity 'System.Text.Json, Version=8.0.0.0, Culture=neutral, PublicKeyToken=cc7b13ffcd2ddd51' of 'System.Text.Json', you may need to supply runtime policy



In [82]:
await Expect(loc).Not.ToHaveAttributeAsync("disabled", "");

In [83]:
await page.EvaluateAsync($"document.querySelector('{css_sel}').setAttribute('disabled', 'true')");


warning CS1701: Assuming assembly reference 'System.Text.Json, Version=6.0.0.0, Culture=neutral, PublicKeyToken=cc7b13ffcd2ddd51' used by 'Microsoft.Playwright' matches identity 'System.Text.Json, Version=8.0.0.0, Culture=neutral, PublicKeyToken=cc7b13ffcd2ddd51' of 'System.Text.Json', you may need to supply runtime policy



In [84]:
await Expect(loc).ToHaveAttributeAsync("disabled", "true");

# I wait for the contents of a popup alert and I dismiss it

In [85]:
await page.GotoAsync("https://play1.automationcamp.ir/expected_conditions.html");

In [86]:
var sel = "button#alert_trigger";
var alert_msg = "I am alerting you!";
var badge = "#alert_handled_badge";

In [87]:
assert(await page.Locator(sel).TextContentAsync() == "Show Alert");

In [88]:
assert(!await page.Locator(badge).IsVisibleAsync());

In [89]:
System.EventHandler<Microsoft.Playwright.IDialog> alert_dialog_handler = async (_, dialog) => {
  Console.WriteLine($"Dialog message: {dialog.Message}");
  assert(DialogType.Alert == dialog.Type, "Expected an alert dialog.");
  assert(alert_msg == dialog.Message, "Unexpected dialog message.");
  await dialog.AcceptAsync();
};

In [90]:
 // Set up the dialog event handler
page.Dialog += alert_dialog_handler

In [91]:
// Trigger the alert and handle the dialog
await page.ClickAsync(sel);

// By chance here because Jupyter Notebooks does not handle Async super well if the assertions inside the async for the dialog event handler above
// fail then we will get and error like so
/*
{
	"name": "",
	"message": "",
	"stack": "The Kernel crashed while executing code in the current cell or a previous cell. 
Please review the code in the cell(s) to identify a possible cause of the failure. 
Click <a href='https://aka.ms/vscodeJupyterKernelCrash'>here</a> for more info. 
View Jupyter <a href='command:jupyter.viewOutput'>log</a> for further details."
}
*/
//From a testing perspective this achieves what we want, meaning that it stops the executions and it fails. So it fails when assertions fail
// and it does nothing when they pass. This is the desired behavior for testing.

In [92]:
// await page.Locator(badge).WaitForAsync(new LocatorWaitForOptions { State = WaitForSelectorState.Visible });
await page.Locator(badge).WaitForAsync(new() { State = WaitForSelectorState.Visible });

In [93]:
var alert_badge = page.Locator(badge);

In [94]:
assert((await alert_badge.TextContentAsync()).Contains("Alert"));
assert((await alert_badge.TextContentAsync()).Contains("handled"));

In [95]:
page.Dialog -= alert_dialog_handler;

# I wait for the confirmation popup and I accept it

In [96]:
var sel = "button#prompt_trigger";
var confirmation_msg = "Choose wisely...\nIt's your life!";
var confirm_badge = "#confirm_ok_badge";

In [97]:
await Expect(page.Locator(sel)).ToHaveTextAsync("Show Prompt");

In [98]:
assert(!await page.Locator(confirm_badge).IsVisibleAsync());

In [99]:
string dialog_type = string.Empty;
string dialog_message = string.Empty;

In [100]:
// Define the dialog event handler
System.EventHandler<Microsoft.Playwright.IDialog> confirm_dialog_handler = async (_, dialog) => {
  dialog_type = dialog.Type;
  // isConfirmDialog = DialogType.Confirm == dialog.Type;
  dialog_message = dialog.Message;
  await dialog.AcceptAsync();
};

In [101]:
// Set up the dialog event handler
page.Dialog += confirm_dialog_handler;

In [102]:
await page.ClickAsync(sel);
await Expect(page.Locator(confirm_badge)).ToBeVisibleAsync();

In [103]:
assert(dialog_type == "confirm");
assert(dialog_message == confirmation_msg);

In [104]:
await Expect(page.Locator(confirm_badge)).ToBeVisibleAsync();
await Expect(page.Locator(confirm_badge)).ToContainTextAsync("OK");

In [105]:
page.Dialog -= confirm_dialog_handler;

# I wait for the confirmation popup and I reject it

In [106]:
var sel = "button#prompt_trigger";
var confirmation_msg = "Choose wisely...\nIt's your life!";
var confirm_badge = "#confirm_cancelled_badge";

In [107]:
await page.ReloadAsync();

In [108]:
await Expect(page.Locator(sel)).ToHaveTextAsync("Show Prompt");

In [109]:
assert(!await page.Locator(confirm_badge).IsVisibleAsync());

In [110]:
string dialog_type = string.Empty;
string dialog_message = string.Empty;

In [111]:
System.EventHandler<Microsoft.Playwright.IDialog> confirm_dialog_handler = async (_, dialog) => {
  dialog_message = dialog.Message;
  dialog_type = dialog.Type;
  await dialog.DismissAsync();
};

In [112]:
page.Dialog += confirm_dialog_handler;
await page.ClickAsync(sel);
await Expect(page.Locator(confirm_badge)).ToBeVisibleAsync();
page.Dialog -= confirm_dialog_handler;

In [113]:
assert(dialog_type == "confirm");
assert(dialog_message == confirmation_msg);

In [114]:
await Expect(page.Locator(confirm_badge)).ToBeVisibleAsync();
await Expect(page.Locator(confirm_badge)).ToContainTextAsync("Cancelled");

# I trigger a prompt alert and pass "geia sou" as text TODO also did not work for Playwright
WHICH MEANS THAT WE NEED TO FIND ANOTHER WEBSITE TO TEST THIS as it might have to do with how people are getting their info

In [115]:
// This needs to be translated from Python to C#
/*
# await page.goto('https://testpages.herokuapp.com/styled/alerts/alert-test.html')
# txt = 'geia sou'
# html_sel = '#promptreturn'
# button = '#promptexample'
# prompt_msg = 'I prompt you'
# await expect(page.locator(html_sel)).to_have_text('')
# async with page.expect_event('dialog') as dialog_info: # we have prepared the listener but we are not blocking execution
#   await page.click(button)

# dialog = await dialog_info.value
# assert dialog.type == 'prompt'
# assert dialog.message == prompt_msg
# await dialog.accept(txt)
*/

# I can assert inside a changing iframe

In [116]:
await page.GotoAsync("https://qaplayground.dev/apps/changing-iframe/");
var frame = page.FrameLocator("iframe");
var sel = "#msg";
var elem = frame.Locator(sel);
await elem.WaitForAsync(new() { Timeout = 20000 });
await Expect(frame.Locator(sel)).ToHaveTextAsync("This is the end of the journey");

# I can access a nested iframe and assert in it

In [117]:
await page.GotoAsync("https://qaplayground.dev/apps/iframe/");

In [118]:
await Expect(page.Locator("#frame1")).ToBeVisibleAsync();

In [119]:
var frame = page.FrameLocator("#frame1");

In [120]:
await Expect(frame.Locator("legend")).ToHaveTextAsync("First Level Iframe");
await Expect(frame.Locator("#frame2")).ToBeVisibleAsync();

In [121]:
var inner_frame = frame.FrameLocator("#frame2");

In [122]:
await Expect(inner_frame.Locator("legend")).ToHaveTextAsync("Second Level Iframe");

In [123]:
await Expect(inner_frame.Locator("#msg")).Not.ToBeVisibleAsync();
await Expect(inner_frame.Locator(".btn")).ToBeVisibleAsync();

In [124]:
await inner_frame.Locator(".btn").ClickAsync();
await Expect(inner_frame.Locator("#msg")).ToBeVisibleAsync();
await Expect(inner_frame.Locator("#msg")).ToHaveTextAsync("Button Clicked");

# I can manage new tabs

In [125]:
await page.GotoAsync("https://qaplayground.dev/apps/new-tab/");

In [126]:
//https://playwright.dev/dotnet/docs/api/class-browsercontext
var newPageTask = context.RunAndWaitForPageAsync(async () =>
{
  await page.ClickAsync("#open");
});

In [127]:
// Get the new page and ensure it has fully loaded
var newPage = await newPageTask;
await newPage.WaitForLoadStateAsync(LoadState.Load);

In [128]:
var header = newPage.Locator("h1");

In [129]:
await Expect(header).ToBeVisibleAsync();

In [130]:
await Expect(header).ToHaveTextAsync("Welcome to the new page!");

In [131]:
await newPage.CloseAsync();

# I can manage new windows

In [132]:
await page.GotoAsync("https://qaplayground.dev/apps/popup/");

In [133]:
await Expect(page.Locator("#login")).ToHaveTextAsync("Open");

In [134]:
var popup_page = await context.RunAndWaitForPageAsync(async () => {
  await page.ClickAsync("#login");
});

In [135]:
await popup_page.WaitForLoadStateAsync(LoadState.Load);

In [136]:
await Expect(popup_page.Locator("button")).ToHaveTextAsync("Submit");

In [137]:
assert(context.Pages.Contains(popup_page));

In [138]:
await popup_page.Locator("button").ClickAsync();

In [139]:
assert(!context.Pages.Contains(popup_page));

# I can upload a file

In [140]:
var myfile = "balls.jpg";

In [141]:
await page.GotoAsync("https://qaplayground.dev/apps/upload/");

In [142]:
var sel = "input[type='file']";

In [143]:
await Expect(page.Locator(sel)).ToBeAttachedAsync();

In [144]:
//actually uploading the file
await page.SetInputFilesAsync(sel, myfile);

In [145]:
await Expect(page.Locator("#num-of-files")).ToHaveTextAsync("1 File Selected");

In [146]:
await Expect(page.Locator("figcaption")).ToHaveTextAsync(myfile);

# I can download a file

In [147]:
var filename = "sample.pdf";

In [148]:
System.IO.File.Delete(filename);

In [149]:
bool is_pdf(string filepath) {
  byte[] pdf_header = new byte[] { 0x25, 0x50, 0x44, 0x46, 0x2D };
  byte[] file_header = new byte[5];
  using(var file_stream = new System.IO.FileStream(filepath, System.IO.FileMode.Open)) {
    file_stream.Read(file_header, 0, 5);
  }

  return pdf_header.SequenceEqual(file_header);
}

In [150]:
await page.GotoAsync("https://qaplayground.dev/apps/download/");
await Expect(page.Locator("#file")).ToHaveTextAsync("Download ⏬");

In [151]:
// Use RunAndWaitForDownloadAsync to click and wait for the download
var download = await page.RunAndWaitForDownloadAsync(async () => {
  await page.ClickAsync("#file");
});

In [152]:
await download.SaveAsAsync(filename);

In [153]:
assert(is_pdf(filename));

# I can set the Geolocation to custom coordinates

In [154]:
await page.CloseAsync();
await context.CloseAsync();
await browser.CloseAsync();

In [155]:
var browser = await playwright.Chromium.LaunchAsync(new BrowserTypeLaunchOptions { Headless = false });

In [156]:
//https://playwright.dev/dotnet/docs/api/class-browser#browser-new-context
var context = await browser.NewContextAsync(new BrowserNewContextOptions { 
  ViewportSize = new ViewportSize { Width = 1920, Height = 1080 },
  Geolocation = new Geolocation { Latitude = 37.7749f, Longitude = -122.4194f },
  Permissions = new[] { "geolocation" }
});

In [157]:
var page = await context.NewPageAsync();

In [158]:
await page.GotoAsync("https://qaplayground.dev/apps/geolocation/");

In [159]:
await Expect(page.Locator("#location-info")).ToHaveTextAsync("Click on the button to get your current location");

In [160]:
await page.ClickAsync("#get-location");

In [161]:
await Expect(page.Locator("#location-info")).ToHaveTextAsync("San Francisco, United States");

# I can move forward and backward to pages

In [162]:
await page.GotoAsync("https://qaplayground.dev/apps/links/");

In [163]:
await Expect(page.Locator("xpath=//* >> text='Home'")).ToBeVisibleAsync();
await Expect(page.Locator("xpath=//* >> text='Contact'")).ToBeVisibleAsync();
await Expect(page.Locator("#title")).ToHaveCountAsync(0);

In [164]:
await page.ClickAsync("div#nav > a:nth-of-type(5)")

In [165]:
await Expect(page.Locator("#title")).ToHaveTextAsync("Welcome to the Contact Page");
await Expect(page.Locator("xpath=//* >> text='Contact'")).Not.ToBeAttachedAsync();

In [166]:
await page.GoBackAsync();

In [167]:
await Expect(page.Locator("xpath=//* >> text='Home'")).ToBeVisibleAsync();
await Expect(page.Locator("xpath=//* >> text='Contact'")).ToBeVisibleAsync();
await Expect(page.Locator("#title")).ToHaveCountAsync(0);

In [168]:
await page.GoForwardAsync();

In [169]:
await Expect(page.Locator("#title")).ToHaveTextAsync("Welcome to the Contact Page");
await Expect(page.Locator("xpath=//* >> text='Contact'")).Not.ToBeAttachedAsync();

# I can get, set, clear and save Cookies of a webpage

In [170]:
using System.Text.Json;

In [171]:
var filename = "cookies.json";
System.IO.File.Delete(filename);

In [172]:
await page.GotoAsync("https://www.saucedemo.com/");

In [173]:
await page.Locator("#user-name").FillAsync("standard_user");
await page.Locator("#password").FillAsync("secret_sauce");

In [174]:
await page.ClickAsync("#login-button");

In [175]:
await page.WaitForURLAsync("https://www.saucedemo.com/inventory.html");
assert(page.Url == "https://www.saucedemo.com/inventory.html");

In [176]:
var title = await page.Locator(".title").InnerTextAsync();
assert(title == "Products");

In [177]:
//save cookies to a file
var cookies = await context.CookiesAsync();
var cookiesJson = JsonSerializer.Serialize(cookies);
await File.WriteAllTextAsync(filename, cookiesJson);
// using(var file_stream = new System.IO.FileStream(filename, System.IO.FileMode.Create)) {
//   await JsonSerializer.SerializeAsync(file_stream, cookies);
// }

In [178]:
await context.ClearCookiesAsync();

In [179]:
await page.ReloadAsync();

In [180]:
//after reload we are not in inventory page anymore, rather we have been redirected to the login page
await page.WaitForURLAsync("https://www.saucedemo.com/");

In [181]:
using System.IO;

In [182]:
// using(var file_stream = new System.IO.FileStream(filename, System.IO.FileMode.Open)) {
//   var cookies_loaded = await JsonSerializer.DeserializeAsync<Cookie[]>(file_stream);
//   await context.AddCookiesAsync(cookies_loaded);
// }

In [183]:
// Load cookies from the file and add them to the context
// Load cookies from the file and add them to the context
var loadedCookiesJson = await File.ReadAllTextAsync("cookies.json");
var loadedCookies = JsonSerializer.Deserialize<IEnumerable<Cookie>>(loadedCookiesJson);
await context.AddCookiesAsync(loadedCookies);

In [184]:
await page.GotoAsync("https://www.saucedemo.com/inventory.html");
await page.WaitForURLAsync("https://www.saucedemo.com/inventory.html");

In [185]:
var title = await page.Locator(".title").InnerTextAsync();
assert(title == "Products");

# END

In [186]:
await page.CloseAsync();
await context.CloseAsync();
await browser.CloseAsync();
playwright.Dispose();

- Playwright has built-in parallelization support, whereas Selenium requires a third-party tool.
- Playwright executes faster than Selenium.
- Selenium doesn't support features like detailed reporting and video recording, while Playwright provides built-in support.

====

- Selenium can be used in real devices and remote servers, while Playwright doesn't offer this option.
- Selenium supports more browsers and programming languages than its opponent.